In [3]:
import requests

In [1]:
r = requests.get("https://hhinternet.blob.core.windows.net/wait-times/testing-wait-times.pdf")

In [11]:
with open('covid_wait_time.pdf', 'wb') as f:
    f.write(r.content)
f.close()

In [46]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import HTMLConverter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import BytesIO
import re
import csv

def convert_pdf_to_html(path):
    rsrcmgr = PDFResourceManager()
    retstr = BytesIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = HTMLConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    with open(path, 'rb') as fp:
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ""
        maxpages = 0 #is for all
        caching = True
        pagenos=set()
        for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
            interpreter.process_page(page)
    fp.close()
    device.close()
    mystr = retstr.getvalue()
    retstr.close()
    return mystr

In [51]:
#from pdfminer.high_level import extract_text_to_fp

html = convert_pdf_to_html('covid_wait_time.pdf')
with open('covid_wait_time.html', 'wb') as f:
    f.write(html)
f.close()

In [52]:
from bs4 import BeautifulSoup
bs = BeautifulSoup(html)

In [63]:
output = []
for div in bs.find_all('div'):
    spans = div.find_all('span')
    if len(spans) == 2:
        output.append([i.text.replace('\n', '').replace('(cid:415)', 'ti') for i in spans])

In [66]:
import pandas as pd
df = pd.DataFrame.from_records(output, columns=['name', 'wait'])
df.to_csv('wait_times.csv')